In [1]:
'''
audio covid-19  AI

reference: https://github.com/virufy/covid

'''

import random
import pandas as pd
import numpy as np
import os
import librosa
import skimage
import cv2

from PIL import Image
import matplotlib as mpl
import librosa.display

from skimage import io
from tqdm.auto import tqdm

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt
import cv2

import PyQt5 
%matplotlib qt

import warnings
warnings.filterwarnings(action='ignore') 

CFG = {
    'SR':16000,   # 16000,
    'N_MFCC':32, # MFCC 벡터를 추출할 개수
    'SEED':41
}

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(CFG['SEED']) # Seed 고정

train_df = pd.read_csv('open/train_data.csv')
test_df = pd.read_csv('open/test_data.csv')

In [2]:
def get_mfcc_feature(df, data_type, save_path):
    '''
    
    mfcc feature .csv파일로 저장
    mfcc image .png파일로 저장
    
    '''
    
    cmap_name = 'magma'
    cm_conv = mpl.cm.get_cmap(cmap_name)    
    
    # Data Folder path
    root_folder = './open'
    
    if data_type=='train':
        a = df['covid19']
        # print(a[0])
    
    features = []
    for uid in tqdm(df['id']):
        root_path = os.path.join(root_folder, data_type)
        path = os.path.join(root_path, str(uid).zfill(5)+'.wav')

        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=CFG['SR'])
        
        # librosa패키지를 사용하여 mfcc 추출
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CFG['N_MFCC'])
        
        melspec = librosa.feature.melspectrogram(y=y, sr=sr)
                
        if True:                
            
            s_db = librosa.power_to_db(melspec, ref=np.max)
            y2, ints = librosa.effects.trim(y, top_db=30) 
                       
#             print(ints)
            
            pad = 0.25*sr
            
            start=int(max(ints[0]-pad, 0))
            end=int(min(ints[1]+pad, len(y)))
            
            y3 = y[start:end]
            
            chunk = 4.09 # 128로 맞추기 위함.
            y3_ = y3[:np.floor(chunk*sr).astype(int)]
            y4 = np.zeros(int(sr*chunk))
            y4[:min(len(y4), len(y3_))] = y3_[:min(len(y4), len(y3_))]
            
#             melspec2 = librosa.feature.melspectrogram(y=y2, sr=sr)
#             s_db2 = librosa.power_to_db(melspec2, ref=np.max)
            
#             melspec3 = librosa.feature.melspectrogram(y=y3, sr=sr)
#             s_db3 = librosa.power_to_db(melspec3, ref=np.max)            
            
            melspec4 = librosa.feature.melspectrogram(y=y4, sr=sr)
            s_db4 = librosa.power_to_db(melspec4, ref=np.max)
            mfcc4 = librosa.feature.mfcc(y=y4, sr=sr, n_mfcc=CFG['N_MFCC'])
            
            print(uid, y.shape, y4.shape, mfcc.shape, melspec4.shape)
            
            s_db4_n = (s_db4 - np.min(s_db4)) / (np.max(s_db4)-np.min(s_db4))
            im = cm_conv(s_db4_n).astype(float)
            im = np.uint8(im * 255)
            im2 = Image.fromarray(im)
            
            # png파일로 저장
            if data_type=='train':
                im2.save('open/png2/train/mfcc_'+ str(uid).zfill(5) +'.png')
            elif data_type=='test':
                im2.save('open/png2/test/mfcc_'+str(uid).zfill(5)+'.png')
            
            if False:
                plt.figure()
                plt.subplot(421)
                plt.plot(y)
                plt.title(f'id={uid}, covid19={a[uid-1]}')

                plt.subplot(422)
                plt.imshow(s_db, cmap='magma')

                plt.subplot(423)
                plt.plot(y2)

                plt.subplot(424)
                plt.imshow(s_db2, cmap='magma')

                plt.subplot(425)
                plt.plot(y3)

                plt.subplot(426)
                plt.imshow(s_db3, cmap='magma')

                plt.subplot(427)
                plt.plot(y4)

                plt.subplot(428)
                plt.imshow(s_db4, cmap='magma')            

                plt.show()
         

        y_feature = []
        
        # 추출된 MFCC들의 평균을 Feature로 사용
        
        for e in mfcc4:
            y_feature.append(np.mean(e))
        features.append(y_feature)
    
    
    
    
    # 기존의 자가진단 정보를 담은 데이터프레임에 추출된 오디오 Feature를 추가
    mfcc_df = pd.DataFrame(features, columns=['mfcc_'+str(x) for x in range(1,CFG['N_MFCC']+1)])
    df = pd.concat([df, mfcc_df], axis=1)
    df.to_csv(save_path, index=False)
    print('Done.')
    

In [ ]:
# spectrogram 영상도 저장.
get_mfcc_feature(train_df, 'train', './train_mfcc_data2.csv')

In [3]:
# spectrogram 영상도 저장.
get_mfcc_feature(test_df, 'test', './test_mfcc_data2.csv')

  0%|          | 0/5732 [00:00<?, ?it/s]

3806 (156480,) (65440,) (32, 306) (128, 128)
3807 (156480,) (65440,) (32, 306) (128, 128)
3808 (159360,) (65440,) (32, 312) (128, 128)
3809 (68160,) (65440,) (32, 134) (128, 128)
3810 (156480,) (65440,) (32, 306) (128, 128)
3811 (158400,) (65440,) (32, 310) (128, 128)
3812 (130560,) (65440,) (32, 256) (128, 128)
3813 (131520,) (65440,) (32, 257) (128, 128)
3814 (156480,) (65440,) (32, 306) (128, 128)
3815 (93120,) (65440,) (32, 182) (128, 128)
3816 (56640,) (65440,) (32, 111) (128, 128)
3817 (90240,) (65440,) (32, 177) (128, 128)
3818 (159360,) (65440,) (32, 312) (128, 128)
3819 (106560,) (65440,) (32, 209) (128, 128)
3820 (159360,) (65440,) (32, 312) (128, 128)
3821 (158400,) (65440,) (32, 310) (128, 128)
3822 (159360,) (65440,) (32, 312) (128, 128)
3823 (157440,) (65440,) (32, 308) (128, 128)
3824 (26880,) (65440,) (32, 53) (128, 128)
3825 (155520,) (65440,) (32, 304) (128, 128)
3826 (157440,) (65440,) (32, 308) (128, 128)
3827 (156480,) (65440,) (32, 306) (128, 128)
3828 (149760,) (

3990 (157440,) (65440,) (32, 308) (128, 128)
3991 (114240,) (65440,) (32, 224) (128, 128)
3992 (157440,) (65440,) (32, 308) (128, 128)
3993 (157440,) (65440,) (32, 308) (128, 128)
3994 (148800,) (65440,) (32, 291) (128, 128)
3995 (158400,) (65440,) (32, 310) (128, 128)
3996 (159360,) (65440,) (32, 312) (128, 128)
3997 (48000,) (65440,) (32, 94) (128, 128)
3998 (157440,) (65440,) (32, 308) (128, 128)
3999 (158400,) (65440,) (32, 310) (128, 128)
4000 (76800,) (65440,) (32, 151) (128, 128)
4001 (54720,) (65440,) (32, 107) (128, 128)
4002 (52800,) (65440,) (32, 104) (128, 128)
4003 (16320,) (65440,) (32, 32) (128, 128)
4004 (158400,) (65440,) (32, 310) (128, 128)
4005 (53760,) (65440,) (32, 106) (128, 128)
4006 (159360,) (65440,) (32, 312) (128, 128)
4007 (158400,) (65440,) (32, 310) (128, 128)
4008 (106560,) (65440,) (32, 209) (128, 128)
4009 (156480,) (65440,) (32, 306) (128, 128)
4010 (216000,) (65440,) (32, 422) (128, 128)
4011 (157440,) (65440,) (32, 308) (128, 128)
4012 (65280,) (654

4174 (45120,) (65440,) (32, 89) (128, 128)
4175 (72000,) (65440,) (32, 141) (128, 128)
4176 (158400,) (65440,) (32, 310) (128, 128)
4177 (120000,) (65440,) (32, 235) (128, 128)
4178 (157440,) (65440,) (32, 308) (128, 128)
4179 (156480,) (65440,) (32, 306) (128, 128)
4180 (159360,) (65440,) (32, 312) (128, 128)
4181 (159360,) (65440,) (32, 312) (128, 128)
4182 (157440,) (65440,) (32, 308) (128, 128)
4183 (159360,) (65440,) (32, 312) (128, 128)
4184 (152640,) (65440,) (32, 299) (128, 128)
4185 (44160,) (65440,) (32, 87) (128, 128)
4186 (153600,) (65440,) (32, 301) (128, 128)
4187 (140160,) (65440,) (32, 274) (128, 128)
4188 (158400,) (65440,) (32, 310) (128, 128)
4189 (116160,) (65440,) (32, 227) (128, 128)
4190 (156480,) (65440,) (32, 306) (128, 128)
4191 (159360,) (65440,) (32, 312) (128, 128)
4192 (158400,) (65440,) (32, 310) (128, 128)
4193 (157440,) (65440,) (32, 308) (128, 128)
4194 (157440,) (65440,) (32, 308) (128, 128)
4195 (158400,) (65440,) (32, 310) (128, 128)
4196 (159360,) 

4358 (136320,) (65440,) (32, 267) (128, 128)
4359 (158400,) (65440,) (32, 310) (128, 128)
4360 (159360,) (65440,) (32, 312) (128, 128)
4361 (121920,) (65440,) (32, 239) (128, 128)
4362 (155520,) (65440,) (32, 304) (128, 128)
4363 (157440,) (65440,) (32, 308) (128, 128)
4364 (62400,) (65440,) (32, 122) (128, 128)
4365 (158400,) (65440,) (32, 310) (128, 128)
4366 (96960,) (65440,) (32, 190) (128, 128)
4367 (33600,) (65440,) (32, 66) (128, 128)
4368 (155520,) (65440,) (32, 304) (128, 128)
4369 (158400,) (65440,) (32, 310) (128, 128)
4370 (156480,) (65440,) (32, 306) (128, 128)
4371 (158400,) (65440,) (32, 310) (128, 128)
4372 (109440,) (65440,) (32, 214) (128, 128)
4373 (21120,) (65440,) (32, 42) (128, 128)
4374 (156480,) (65440,) (32, 306) (128, 128)
4375 (149760,) (65440,) (32, 293) (128, 128)
4376 (159360,) (65440,) (32, 312) (128, 128)
4377 (68160,) (65440,) (32, 134) (128, 128)
4378 (112320,) (65440,) (32, 220) (128, 128)
4379 (85440,) (65440,) (32, 167) (128, 128)
4380 (158400,) (65

...

8784 (70080,) (65440,) (32, 137) (128, 128)
8785 (125760,) (65440,) (32, 246) (128, 128)
8786 (158400,) (65440,) (32, 310) (128, 128)
8787 (75840,) (65440,) (32, 149) (128, 128)
8788 (154560,) (65440,) (32, 302) (128, 128)
8789 (74880,) (65440,) (32, 147) (128, 128)
8790 (151680,) (65440,) (32, 297) (128, 128)
8791 (87360,) (65440,) (32, 171) (128, 128)
8792 (84480,) (65440,) (32, 166) (128, 128)
8793 (52800,) (65440,) (32, 104) (128, 128)
8794 (157440,) (65440,) (32, 308) (128, 128)
8795 (158400,) (65440,) (32, 310) (128, 128)
8796 (156480,) (65440,) (32, 306) (128, 128)
8797 (109440,) (65440,) (32, 214) (128, 128)
8798 (156480,) (65440,) (32, 306) (128, 128)
8799 (159360,) (65440,) (32, 312) (128, 128)
8800 (111360,) (65440,) (32, 218) (128, 128)
8801 (28800,) (65440,) (32, 57) (128, 128)
8802 (157440,) (65440,) (32, 308) (128, 128)
8803 (157440,) (65440,) (32, 308) (128, 128)
8804 (156480,) (65440,) (32, 306) (128, 128)
8805 (159360,) (65440,) (32, 312) (128, 128)
8806 (159360,) (65

8968 (159360,) (65440,) (32, 312) (128, 128)
8969 (157440,) (65440,) (32, 308) (128, 128)
8970 (159360,) (65440,) (32, 312) (128, 128)
8971 (156480,) (65440,) (32, 306) (128, 128)
8972 (130560,) (65440,) (32, 256) (128, 128)
8973 (117120,) (65440,) (32, 229) (128, 128)
8974 (156480,) (65440,) (32, 306) (128, 128)
8975 (120960,) (65440,) (32, 237) (128, 128)
8976 (153600,) (65440,) (32, 301) (128, 128)
8977 (155520,) (65440,) (32, 304) (128, 128)
8978 (158400,) (65440,) (32, 310) (128, 128)
8979 (156480,) (65440,) (32, 306) (128, 128)
8980 (31680,) (65440,) (32, 62) (128, 128)
8981 (158400,) (65440,) (32, 310) (128, 128)
8982 (143040,) (65440,) (32, 280) (128, 128)
8983 (105600,) (65440,) (32, 207) (128, 128)
8984 (157440,) (65440,) (32, 308) (128, 128)
8985 (153600,) (65440,) (32, 301) (128, 128)
8986 (157440,) (65440,) (32, 308) (128, 128)
8987 (153600,) (65440,) (32, 301) (128, 128)
8988 (79680,) (65440,) (32, 156) (128, 128)
8989 (81600,) (65440,) (32, 160) (128, 128)
8990 (154560,)

9153 (157440,) (65440,) (32, 308) (128, 128)
9154 (44160,) (65440,) (32, 87) (128, 128)
9155 (91200,) (65440,) (32, 179) (128, 128)
9156 (157440,) (65440,) (32, 308) (128, 128)
9157 (159360,) (65440,) (32, 312) (128, 128)
9158 (158400,) (65440,) (32, 310) (128, 128)
9159 (61440,) (65440,) (32, 121) (128, 128)
9160 (67200,) (65440,) (32, 132) (128, 128)
9161 (158400,) (65440,) (32, 310) (128, 128)
9162 (38400,) (65440,) (32, 76) (128, 128)
9163 (118080,) (65440,) (32, 231) (128, 128)
9164 (156480,) (65440,) (32, 306) (128, 128)
9165 (156480,) (65440,) (32, 306) (128, 128)
9166 (83520,) (65440,) (32, 164) (128, 128)
9167 (156480,) (65440,) (32, 306) (128, 128)
9168 (130560,) (65440,) (32, 256) (128, 128)
9169 (102720,) (65440,) (32, 201) (128, 128)
9170 (157440,) (65440,) (32, 308) (128, 128)
9171 (157440,) (65440,) (32, 308) (128, 128)
9172 (158400,) (65440,) (32, 310) (128, 128)
9173 (70080,) (65440,) (32, 137) (128, 128)
9174 (159360,) (65440,) (32, 312) (128, 128)
9175 (156480,) (654

9337 (156480,) (65440,) (32, 306) (128, 128)
9338 (153600,) (65440,) (32, 301) (128, 128)
9339 (151680,) (65440,) (32, 297) (128, 128)
9340 (60480,) (65440,) (32, 119) (128, 128)
9341 (98880,) (65440,) (32, 194) (128, 128)
9342 (159360,) (65440,) (32, 312) (128, 128)
9343 (120960,) (65440,) (32, 237) (128, 128)
9344 (159360,) (65440,) (32, 312) (128, 128)
9345 (157440,) (65440,) (32, 308) (128, 128)
9346 (159360,) (65440,) (32, 312) (128, 128)
9347 (157440,) (65440,) (32, 308) (128, 128)
9348 (159360,) (65440,) (32, 312) (128, 128)
9349 (149760,) (65440,) (32, 293) (128, 128)
9350 (96000,) (65440,) (32, 188) (128, 128)
9351 (155520,) (65440,) (32, 304) (128, 128)
9352 (127680,) (65440,) (32, 250) (128, 128)
9353 (117120,) (65440,) (32, 229) (128, 128)
9354 (118080,) (65440,) (32, 231) (128, 128)
9355 (157440,) (65440,) (32, 308) (128, 128)
9356 (159360,) (65440,) (32, 312) (128, 128)
9357 (156480,) (65440,) (32, 306) (128, 128)
9358 (152640,) (65440,) (32, 299) (128, 128)
9359 (226560,

9520 (156480,) (65440,) (32, 306) (128, 128)
9521 (156480,) (65440,) (32, 306) (128, 128)
9522 (156480,) (65440,) (32, 306) (128, 128)
9523 (159360,) (65440,) (32, 312) (128, 128)
9524 (157440,) (65440,) (32, 308) (128, 128)
9525 (159360,) (65440,) (32, 312) (128, 128)
9526 (159360,) (65440,) (32, 312) (128, 128)
9527 (159360,) (65440,) (32, 312) (128, 128)
9528 (158400,) (65440,) (32, 310) (128, 128)
9529 (72000,) (65440,) (32, 141) (128, 128)
9530 (88320,) (65440,) (32, 173) (128, 128)
9531 (144000,) (65440,) (32, 282) (128, 128)
9532 (159360,) (65440,) (32, 312) (128, 128)
9533 (102720,) (65440,) (32, 201) (128, 128)
9534 (100800,) (65440,) (32, 197) (128, 128)
9535 (58560,) (65440,) (32, 115) (128, 128)
9536 (157440,) (65440,) (32, 308) (128, 128)
9537 (159360,) (65440,) (32, 312) (128, 128)
Done.
